In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.diagnostics import cross_validation
from sklearn.preprocessing import StandardScaler
from prophet.diagnostics import performance_metrics

In [149]:
def modelimproving(path, lookback='1095 days', pred_month=1,logisticcap = 4,):
    index = pred_month-1
    df=pd.read_csv(path)
    df = df.sort_values(by=['ds'])
    df = df.dropna()
    target1 = pd.DataFrame()
    target1['ds'] = pd.to_datetime(df['ds'])
    target1['y'] = df['y']
    m = Prophet(seasonality_mode='multiplicative')
    m.fit(target1)
    df_cv = cross_validation(m, initial=lookback, period='180 days', horizon='360 days')
    df_p = performance_metrics(df_cv, rolling_window=0, monthly=True)
    print('Basic univariable mape:{:0.2f}'.format(df_p['mape'][index]))
    
    target1['cpi']=df['cpi'].shift(pred_month)
    target1['bus_conf']=df['bus_conf'].shift(pred_month)
    target1['con_exp']=df['con_exp'].shift(pred_month)
    target1['real_gdp']=df['real_gdp'].shift(pred_month)
    target1['gov_con_exp']=df['gov_con_exp'].shift(pred_month)
    target1.index = range(len(target1))
    target1 = target1.drop(list(range(0,pred_month)))
    target1.index = range(len(target1))
    m2 = Prophet(seasonality_mode='multiplicative')
    m2.add_regressor('cpi')
    m2.add_regressor('bus_conf')
    m2.add_regressor('con_exp')
    m2.add_regressor('real_gdp')
    m2.add_regressor('gov_con_exp')
    m2.fit(target1)
    df_cv2 = cross_validation(m2, initial=lookback, period='180 days', horizon = '360 days')
    df_p2 = performance_metrics(df_cv2, rolling_window=0, monthly=True)
    print('Regression model mape:{:0.2f}'.format(df_p2['mape'][index])) 
    

    ss = StandardScaler()
    ss.fit(target1[['cpi','bus_conf','con_exp','real_gdp','gov_con_exp']])
    df_ss = pd.DataFrame(ss.transform(target1[['cpi','bus_conf','con_exp','real_gdp','gov_con_exp']]))
    df_ss.columns = ['cpi','bus_conf','con_exp','real_gdp','gov_con_exp']
    target2 = pd.DataFrame()
    target2['ds'] = target1['ds']
    target2['y'] = target1['y']
    target2['cpi']=df_ss['cpi']
    target2['bus_conf']=df_ss['bus_conf']
    target2['con_exp']=df_ss['con_exp']
    target2['real_gdp']=df_ss['real_gdp']
    target2['gov_con_exp']=df_ss['gov_con_exp']
    m3 = Prophet(seasonality_mode='multiplicative')
    m3.add_regressor('cpi')
    m3.add_regressor('bus_conf')
    m3.add_regressor('con_exp')
    m3.add_regressor('real_gdp')
    m3.add_regressor('gov_con_exp')
    m3.fit(target2)
    df_cv3 = cross_validation(m3, initial=lookback, period='180 days', horizon='360 days')
    df_p3 = performance_metrics(df_cv3, rolling_window=0, monthly=True)
    print('StandardScaler Regression model mape:{:0.2f}'.format(df_p3['mape'][index]))     

    target4 = pd.DataFrame()
    target4['ds'] = pd.to_datetime(df['ds'])
    target4['y'] = df['y']
    target4['cap'] = logisticcap
    m4 = Prophet(seasonality_mode='multiplicative',growth='logistic')
    m4.fit(target4)
    df_cv4 = cross_validation(m4, initial=lookback, period='180 days', horizon='360 days')
    df_p4 = performance_metrics(df_cv4, rolling_window=0, monthly=True)
    print('Logistic Kernel model mape:{:0.2f}'.format(df_p4['mape'][index]))
    
    m5 = Prophet(seasonality_mode='multiplicative',growth='logistic')
    target1['cap'] = logisticcap
    m5.add_regressor('cpi')
    m5.add_regressor('bus_conf')
    m5.add_regressor('con_exp')
    m5.add_regressor('real_gdp')
    m5.add_regressor('gov_con_exp')
    m5.fit(target1)
    df_cv5 = cross_validation(m5, initial=lookback, period='180 days', horizon='360 days')
    df_p5 = performance_metrics(df_cv5, rolling_window=0, monthly=True)
    print('Regression + Logistic Kernel model mape:{:0.2f}'.format(df_p5['mape'][index]))    
    
    return df_p,df_p2,df_p3,df_p4,df_p5, df_cv,df_cv2,df_cv3,df_cv4, df_cv5

## TEST

In [150]:
Basic, regression, ss_regression, Logistic, regression_logistic, df1,df2,df3,df4, df5 = modelimproving(path='data_short_term.csv', 
                                                            lookback='1095 days', 
                                                            pred_month=1,
                                                            logisticcap=4,)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 98 forecasts with cutoffs between 1972-05-17 00:00:00 and 2020-03-06 00:00:00


  0%|          | 0/98 [00:00<?, ?it/s]

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Basic univariable mape:2.10


INFO:prophet:Making 98 forecasts with cutoffs between 1972-05-17 00:00:00 and 2020-03-06 00:00:00


...

  0%|          | 0/98 [00:00<?, ?it/s]

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 98 forecasts with cutoffs between 1972-05-17 00:00:00 and 2020-03-06 00:00:00


Logistic Kernel model mape:1.05


  0%|          | 0/98 [00:00<?, ?it/s]

Regression + Logistic Kernel model mape:2.04


In [154]:
def run_dump_model(path, out_file, lookback='1095 days', pred_month=1,logisticcap = 4,):
    Basic, regression, ss_regression, Logistic, regression_logistic, df1,df2, df3, df4, df5 = modelimproving(path, 
                                                                                                             lookback=lookback, 
                                                                                                             pred_month=pred_month,
                                                                                                             logisticcap=logisticcap,)
    Basic, regression, ss_regression, Logistic, regression_logistic, df1,df2, df3, df4, df5 = Basic.to_json(), regression.to_json(), ss_regression.to_json(), Logistic.to_json(), regression_logistic.to_json(), df1.to_json(),df2.to_json(), df3.to_json(), df4.to_json(), df5.to_json()
    temp = {"Basic":Basic, 
            "regression":regression,
            "ss_regression":ss_regression,
            "Logistic":Logistic,
            "regression_logistic":regression_logistic,
            "Basic_df":df1, 
            "regression_df":df2,
            "ss_regression_df":df3,
            "Logistic_df":df4,
            "regression_logistic_df":df5}
    jsonString = json.dumps(temp)
    jsonFile = open(out_file, "w")
    jsonFile.write(jsonString)
    jsonFile.close()
    return 0

In [155]:
one_month = ['data_short_term.csv','one_month.json','1095 days',1,4]
two_month = ['data_short_term.csv','two_month.json','2555 days',2,4]
three_month = ['data_short_term.csv','three_month.json','2920 days',3,4]
four_month = ['data_short_term.csv','four_month.json','2920 days',4,4]
five_month = ['data_short_term.csv','five_month.json','2920 days',5,4]
six_month = ['data_short_term.csv','six_month.json','2920 days',6,4]
seven_month = ['data_short_term.csv','seven_month.json','2555 days',7,4]
eight_month = ['data_short_term.csv','eight_month.json','2920 days',8,4]
nine_month = ['data_short_term.csv','nine_month.json','2920 days',9,4]
ten_month = ['data_short_term.csv','ten_month.json','2920 days',10,4]
eleven_month = ['data_short_term.csv','eleven_month.json','2920 days',11,4]
twelve_month = ['data_short_term.csv','twelve_month.json','2920 days',12,4]

In [156]:
for i in [one_month,two_month,three_month,four_month,
          five_month,six_month,seven_month,eight_month,nine_month]:
    run_dump_model(*i)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 98 forecasts with cutoffs between 1972-05-17 00:00:00 and 2020-03-06 00:00:00


  0%|          | 0/98 [00:00<?, ?it/s]

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Basic univariable mape:2.10


INFO:prophet:Making 98 forecasts with cutoffs between 1972-05-17 00:00:00 and 2020-03-06 00:00:00


...

  0%|          | 0/88 [00:00<?, ?it/s]

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Making 87 forecasts with cutoffs between 1977-10-18 00:00:00 and 2020-03-06 00:00:00


Logistic Kernel model mape:1.46


  0%|          | 0/87 [00:00<?, ?it/s]

Regression + Logistic Kernel model mape:1.55


In [119]:
with open("temp.json", 'r') as j:
     contents = json.loads(j.read())
pd.DataFrame(eval(contents['Basic']))

,horizon,mse,rmse,mae,mape,mdape,smape,coverage
0,1,0.268654,0.518319,0.250552,4.485375,1.032251,1.345946,0.724490
1,2,0.429959,0.655712,0.290343,2.793378,1.018619,1.371282,0.734694
2,3,0.283004,0.531981,0.282314,2.111271,1.014305,1.393683,0.653061
3,4,0.304222,0.551563,0.297259,2.009763,1.033186,1.422348,0.622449
4,5,0.293695,0.541937,0.292002,11.914070,1.103263,1.453125,0.612245
5,6,0.270349,0.519951,0.272291,2.239705,1.082883,1.427429,0.663265
6,7,0.254296,0.504278,0.257949,4.815805,1.061725,1.405532,0.704082
7,8,0.267511,0.517214,0.275569,3.033553,1.071781,1.475481,0.653061
8,9,0.286103,0.534886,0.289956,2.134263,1.067842,1.446899,0.602041
9,10,0.314953,0.561207,0.316714,2.209043,1.150039,1.501397,0.581633
